# ExperimentLGNR
### Take the dataset where the negatives comprise of random genomic data (without repeats) sampled for length and gc content.
### Train a single separable convolutional classifier on triplets and evaluate

## Imports

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
# # Credit: https://stackoverflow.com/questions/34199233/how-to-prevent-tensorflow-from-allocating-the-totality-of-a-gpu-memory
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [2]:
import data
from tensorflow import keras
import nets
from loaders import PremadeTripletClassifierSequence

## Files

In [3]:
# file_name = '/home/zakaria/Data/IdentityData/ShortTrain/60.fa'#file_name = '../Data/sequences.fa'

data_dir = '../Data/Datasets/LGNR/'


train_sequence = f'{data_dir}/train_sequences.fa'
valid_sequence = f'{data_dir}/valid_sequences.fa'
test_sequence = f'{data_dir}/test_sequences.fa'

reverse_train_sequence = f'{data_dir}/reverse_train_sequences.fa'
reverse_valid_sequence = f'{data_dir}/reverse_valid_sequences.fa'
reverse_test_sequence = f'{data_dir}/reverse_test_sequences.fa'

train_triplet_dis = f'{data_dir}/train_triplet_dis.npy'
valid_triplet_dis = f'{data_dir}/valid_triplet_dis.npy'
test_triplet_dis = f'{data_dir}/test_triplet_dis.npy'

train_triplet_sim = f'{data_dir}/train_triplet_sim.npy'
valid_triplet_sim = f'{data_dir}/valid_triplet_sim.npy'
test_triplet_sim = f'{data_dir}/test_triplet_sim.npy'

## Parameters

In [4]:
codings_size=100 
exp_filter_num=4
exp_filter_1d_size= 11

allow_reverse = True
max_len = 600

learning_rate = 0.1
momentum = 0.9
is_nesterov = True
patience = 20
monitor = 'val_crm_f1_score'
start_from_epoch = 10

## Loading Data

In [5]:
train = data.FantomToOneHotConverter(train_sequence, 0, max_len).seq_matrix
reverse_train = data.FantomToOneHotConverter(reverse_train_sequence, 0, max_len).seq_matrix if allow_reverse else None

In [6]:
valid = data.FantomToOneHotConverter(valid_sequence, 0, max_len).seq_matrix
reverse_valid = data.FantomToOneHotConverter(reverse_valid_sequence, 0, max_len).seq_matrix if allow_reverse else None

In [7]:
# test = data.FantomToOneHotConverter(test_sequence, 0, max_len).seq_matrix
# reverse_test = data.FantomToOneHotConverter(reverse_test_sequence, 0, max_len).seq_matrix if allow_reverse else None

In [8]:
train_seq = PremadeTripletClassifierSequence(train, train_triplet_sim, train_triplet_dis, batch_size = 1024, reverse_x_in =reverse_train)
valid_seq = PremadeTripletClassifierSequence(valid, valid_triplet_sim, valid_triplet_dis, batch_size = 1024, reverse_x_in =reverse_valid)
# test_seq = PremadeTripletClassifierSequence(test, test_triplet_sim, test_triplet_dis, batch_size = 1024, reverse_x_in =reverse_test)

In [9]:
for x_batch, y_batch in train_seq:
    _, d1, d2, d3 = x_batch.shape

## Creating Model

In [10]:
model = nets.make_conv_classifier(codings_size, (d1, d2, d3), exp_filter_1d_size, filter_num=exp_filter_num)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4, 600, 3)]       0         
                                                                 
 masking (Masking)           (None, 4, 600, 3)         0         
                                                                 
 separable_conv2d (Separable  (None, 1, 598, 4)        48        
 Conv2D)                                                         
                                                                 
 batch_normalization (BatchN  (None, 1, 598, 4)        16        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 1, 598, 4)         0         
                                                                 
 reshape (Reshape)           (None, 598, 4)            0     

In [11]:
opt = keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum, nesterov=is_nesterov) # 0.1
model.compile(loss='mse', metrics=['accuracy', nets.crm_specificity, tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), nets.crm_f1_score], optimizer=opt) #  

In [12]:
early_stopping = keras.callbacks.EarlyStopping(patience=patience, min_delta=1/100000, restore_best_weights=True, monitor=monitor, start_from_epoch = start_from_epoch, mode='max') #val_precision

In [13]:
def exponential_decay_fn(epoch, lr):
    '''
    This function decreases the learning rate according to the epoch
    '''
    return lr*0.1**(1/100)

lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)

## Training Model

In [14]:
model.fit(train_seq, epochs=500, validation_data=valid_seq, workers=26, callbacks=[early_stopping, lr_scheduler]) #,workers=26

Epoch 1/500
72/72 [==============================] - 15s 160ms/step - loss: 0.2361 - accuracy: 0.6006 - crm_specificity: 0.6396 - recall: 0.5618 - precision: 0.6089 - crm_f1_score: 0.5842 - val_loss: 0.2534 - val_accuracy: 0.5008 - val_crm_specificity: 1.0000 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - val_crm_f1_score: 0.0000e+00 - lr: 0.0977
Epoch 2/500
72/72 [==============================] - 11s 156ms/step - loss: 0.2181 - accuracy: 0.6427 - crm_specificity: 0.6704 - recall: 0.6151 - precision: 0.6511 - crm_f1_score: 0.6324 - val_loss: 0.2567 - val_accuracy: 0.5158 - val_crm_specificity: 0.9932 - val_recall: 0.0385 - val_precision: 0.8506 - val_crm_f1_score: 0.0738 - lr: 0.0955
Epoch 3/500
72/72 [==============================] - 12s 158ms/step - loss: 0.2163 - accuracy: 0.6481 - crm_specificity: 0.6663 - recall: 0.6302 - precision: 0.6535 - crm_f1_score: 0.6414 - val_loss: 0.2589 - val_accuracy: 0.5266 - val_crm_specificity: 0.9899 - val_recall: 0.0639 - val_precision: 

In [15]:
model.save("ExperimentLGNRModel")

INFO:tensorflow:Assets written to: ExperimentLGNRModel/assets


INFO:tensorflow:Assets written to: ExperimentLGNRModel/assets


## Evaluating Model

In [16]:
nets.print_results(train_seq, model)

72/72 [==============================] - 9s 119ms/step - loss: 0.2113 - accuracy: 0.6623 - crm_specificity: 0.7758 - recall: 0.5488 - precision: 0.7099 - crm_f1_score: 0.6187
0.2113,66.2,77.29,55.09,70.82,61.94


In [17]:
nets.print_results(valid_seq, model)

15/15 [==============================] - 2s 118ms/step - loss: 0.2148 - accuracy: 0.6512 - crm_specificity: 0.7755 - recall: 0.5267 - precision: 0.7018 - crm_f1_score: 0.6017
0.2158,64.93,77.35,52.49,69.85,59.93
